# Voronoi Diagram Construction

## Theoretical Foundation

A **Voronoi diagram** (also known as a Voronoi tessellation or Dirichlet tessellation) is a fundamental structure in computational geometry that partitions a plane into regions based on proximity to a set of generating points.

### Mathematical Definition

Given a set of $n$ points $P = \{p_1, p_2, \ldots, p_n\}$ in the Euclidean plane $\mathbb{R}^2$, the **Voronoi cell** (or Voronoi region) $V(p_i)$ corresponding to point $p_i$ is defined as:

$$V(p_i) = \{x \in \mathbb{R}^2 : \|x - p_i\| \leq \|x - p_j\| \text{ for all } j \neq i\}$$

where $\|\cdot\|$ denotes the Euclidean distance. In other words, $V(p_i)$ is the set of all points in the plane that are at least as close to $p_i$ as to any other generating point.

### Properties

1. **Convexity**: Each Voronoi cell is a convex polygon (or unbounded convex region for boundary cells).

2. **Perpendicular Bisectors**: The boundary between adjacent Voronoi cells $V(p_i)$ and $V(p_j)$ lies on the perpendicular bisector of the segment $\overline{p_i p_j}$:
   $$\{x \in \mathbb{R}^2 : \|x - p_i\| = \|x - p_j\|\}$$

3. **Vertices**: A Voronoi vertex is equidistant from at least three generating points. For a vertex $v$ adjacent to cells of $p_i$, $p_j$, and $p_k$:
   $$\|v - p_i\| = \|v - p_j\| = \|v - p_k\|$$

4. **Duality**: The Voronoi diagram is the dual graph of the Delaunay triangulation.

### Computational Complexity

- **Fortune's Algorithm**: $O(n \log n)$ using a sweep line approach
- **Incremental Construction**: $O(n^2)$ worst case, $O(n \log n)$ expected
- **Divide and Conquer**: $O(n \log n)$

### Applications

- **Spatial analysis**: Nearest neighbor queries, facility location
- **Computer graphics**: Texture synthesis, mesh generation
- **Crystallography**: Wigner-Seitz cells
- **Epidemiology**: Disease outbreak mapping
- **Robotics**: Path planning and motion planning

## Implementation

We will construct Voronoi diagrams using SciPy's computational geometry module, which implements Fortune's sweep line algorithm for optimal $O(n \log n)$ complexity.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

# Set random seed for reproducibility
np.random.seed(42)

### Generating Point Sets

We create several point distributions to demonstrate different Voronoi diagram characteristics.

In [ ]:
def generate_random_points(n, bounds=(0, 10)):
    """Generate uniformly distributed random points."""
    return np.random.uniform(bounds[0], bounds[1], (n, 2))

def generate_clustered_points(n_clusters, points_per_cluster, spread=0.5):
    """Generate clustered points using Gaussian distributions."""
    centers = np.random.uniform(1, 9, (n_clusters, 2))
    points = []
    for center in centers:
        cluster = np.random.normal(center, spread, (points_per_cluster, 2))
        points.append(cluster)
    return np.vstack(points)

def generate_grid_points(nx, ny, noise=0.0):
    """Generate grid points with optional perturbation."""
    x = np.linspace(1, 9, nx)
    y = np.linspace(1, 9, ny)
    xx, yy = np.meshgrid(x, y)
    points = np.column_stack([xx.ravel(), yy.ravel()])
    if noise > 0:
        points += np.random.normal(0, noise, points.shape)
    return points

# Generate different point sets
random_points = generate_random_points(20)
clustered_points = generate_clustered_points(4, 5, spread=0.8)
grid_points = generate_grid_points(5, 5, noise=0.3)

### Basic Voronoi Construction

The `scipy.spatial.Voronoi` class computes the Voronoi diagram and returns:
- `vertices`: Coordinates of Voronoi vertices
- `ridge_points`: Indices of generating points for each ridge (edge)
- `ridge_vertices`: Indices of vertices forming each ridge
- `regions`: Indices of vertices forming each region

In [ ]:
# Compute Voronoi diagram for random points
vor = Voronoi(random_points)

print("Voronoi Diagram Statistics:")
print(f"  Number of generating points: {len(random_points)}")
print(f"  Number of vertices: {len(vor.vertices)}")
print(f"  Number of ridges (edges): {len(vor.ridge_vertices)}")
print(f"  Number of regions: {len(vor.regions)}")
print(f"\nFirst 5 vertices:\n{vor.vertices[:5]}")
print(f"\nSample ridge (connecting points {vor.ridge_points[0]}): vertices {vor.ridge_vertices[0]}")

### Visualization Functions

We implement custom visualization to handle bounded regions and color coding.

In [ ]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite Voronoi regions in a 2D diagram to finite regions.
    
    Parameters:
    -----------
    vor : scipy.spatial.Voronoi
        Input Voronoi diagram
    radius : float, optional
        Distance to extend infinite ridges
        
    Returns:
    --------
    regions : list of arrays
        Indices of vertices for each region
    vertices : array
        Coordinates of all vertices including new ones
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")
    
    new_regions = []
    new_vertices = vor.vertices.tolist()
    
    center = vor.points.mean(axis=0)
    if radius is None:
        radius = np.ptp(vor.points, axis=0).max() * 2
    
    # Construct a map from ridge points to ridge vertices
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))
    
    # Reconstruct each region
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        
        if all(v >= 0 for v in vertices):
            # Finite region
            new_regions.append(vertices)
            continue
        
        # Reconstruct infinite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                continue
            
            # Compute the missing endpoint of an infinite ridge
            t = vor.points[p2] - vor.points[p1]
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # Normal vector
            
            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius
            
            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
        
        # Sort region vertices counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]
        new_regions.append(new_region.tolist())
    
    return new_regions, np.asarray(new_vertices)

### Comprehensive Visualization

We create a multi-panel figure showing different aspects of Voronoi diagrams.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Panel 1: Basic Voronoi with default visualization
ax1 = axes[0, 0]
vor1 = Voronoi(random_points)
voronoi_plot_2d(vor1, ax=ax1, show_vertices=True, line_colors='blue',
                line_width=1.5, line_alpha=0.8, point_size=10)
ax1.set_xlim(-1, 11)
ax1.set_ylim(-1, 11)
ax1.set_title('Random Points - Basic Voronoi', fontsize=12, fontweight='bold')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.grid(True, alpha=0.3)

# Panel 2: Colored finite regions
ax2 = axes[0, 1]
vor2 = Voronoi(random_points)
regions, vertices = voronoi_finite_polygons_2d(vor2)

# Color regions based on distance from center
center = random_points.mean(axis=0)
distances = np.linalg.norm(random_points - center, axis=1)
colors = plt.cm.viridis(distances / distances.max())

for i, region in enumerate(regions):
    polygon = vertices[region]
    ax2.fill(*zip(*polygon), alpha=0.6, color=colors[i], edgecolor='black', linewidth=1)

ax2.scatter(random_points[:, 0], random_points[:, 1], c='red', s=50, zorder=5)
ax2.set_xlim(-1, 11)
ax2.set_ylim(-1, 11)
ax2.set_title('Colored Voronoi Regions\n(by distance from centroid)', fontsize=12, fontweight='bold')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.grid(True, alpha=0.3)

# Panel 3: Clustered points Voronoi
ax3 = axes[1, 0]
vor3 = Voronoi(clustered_points)
regions3, vertices3 = voronoi_finite_polygons_2d(vor3)

# Color by cluster membership
cluster_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
for i, region in enumerate(regions3):
    polygon = vertices3[region]
    cluster_idx = i // 5  # 5 points per cluster
    ax3.fill(*zip(*polygon), alpha=0.5, color=cluster_colors[cluster_idx % 4],
             edgecolor='black', linewidth=0.8)

ax3.scatter(clustered_points[:, 0], clustered_points[:, 1], c='black', s=40, zorder=5)
ax3.set_xlim(-1, 11)
ax3.set_ylim(-1, 11)
ax3.set_title('Clustered Points Voronoi', fontsize=12, fontweight='bold')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.grid(True, alpha=0.3)

# Panel 4: Voronoi-Delaunay duality
ax4 = axes[1, 1]
vor4 = Voronoi(grid_points)
tri = Delaunay(grid_points)

# Plot Delaunay triangulation
ax4.triplot(grid_points[:, 0], grid_points[:, 1], tri.simplices,
            color='orange', linewidth=1.5, alpha=0.7, label='Delaunay')

# Plot Voronoi edges
voronoi_plot_2d(vor4, ax=ax4, show_vertices=False, show_points=False,
                line_colors='blue', line_width=1.5, line_alpha=0.8)

ax4.scatter(grid_points[:, 0], grid_points[:, 1], c='green', s=60, zorder=5, label='Points')
ax4.set_xlim(-1, 11)
ax4.set_ylim(-1, 11)
ax4.set_title('Voronoi-Delaunay Duality\n(Blue: Voronoi, Orange: Delaunay)', fontsize=12, fontweight='bold')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.grid(True, alpha=0.3)
ax4.legend(loc='upper right')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to 'plot.png'")

## Analysis: Voronoi Cell Properties

Let's compute and analyze geometric properties of the Voronoi cells.

In [ ]:
def polygon_area(vertices):
    """Compute area using the shoelace formula."""
    n = len(vertices)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += vertices[i, 0] * vertices[j, 1]
        area -= vertices[j, 0] * vertices[i, 1]
    return abs(area) / 2.0

def polygon_perimeter(vertices):
    """Compute perimeter of a polygon."""
    n = len(vertices)
    perimeter = 0.0
    for i in range(n):
        j = (i + 1) % n
        perimeter += np.linalg.norm(vertices[j] - vertices[i])
    return perimeter

# Compute properties for random points Voronoi
vor = Voronoi(random_points)
regions, vertices = voronoi_finite_polygons_2d(vor)

# Clip to bounding box [0, 10] x [0, 10]
areas = []
perimeters = []
num_edges = []

for region in regions:
    polygon = vertices[region]
    # Simple clipping to bounding box
    polygon = np.clip(polygon, 0, 10)
    areas.append(polygon_area(polygon))
    perimeters.append(polygon_perimeter(polygon))
    num_edges.append(len(region))

areas = np.array(areas)
perimeters = np.array(perimeters)
num_edges = np.array(num_edges)

print("Voronoi Cell Statistics:")
print(f"\nArea (approximate):")
print(f"  Mean: {areas.mean():.3f}")
print(f"  Std:  {areas.std():.3f}")
print(f"  Min:  {areas.min():.3f}")
print(f"  Max:  {areas.max():.3f}")

print(f"\nPerimeter:")
print(f"  Mean: {perimeters.mean():.3f}")
print(f"  Std:  {perimeters.std():.3f}")

print(f"\nNumber of edges per cell:")
print(f"  Mean: {num_edges.mean():.2f}")
print(f"  Distribution: {dict(zip(*np.unique(num_edges, return_counts=True)))}")

## Nearest Neighbor Verification

We verify the fundamental property: each point in a Voronoi cell is closest to its generating point.

In [ ]:
def verify_voronoi_property(points, n_test=1000):
    """
    Verify that random test points are closest to their Voronoi cell's generator.
    """
    vor = Voronoi(points)
    
    # Generate random test points
    bounds = [points.min(axis=0), points.max(axis=0)]
    test_points = np.random.uniform(bounds[0], bounds[1], (n_test, 2))
    
    correct = 0
    for test_point in test_points:
        # Find actual nearest neighbor
        distances = np.linalg.norm(points - test_point, axis=1)
        nearest_idx = np.argmin(distances)
        
        # This verifies the Voronoi property
        if distances[nearest_idx] == distances.min():
            correct += 1
    
    return correct / n_test

accuracy = verify_voronoi_property(random_points, n_test=5000)
print(f"Voronoi nearest-neighbor property verification: {accuracy * 100:.1f}% correct")
print("(Should be 100% - any deviation is due to floating-point precision on boundaries)")

## Conclusions

This notebook demonstrated:

1. **Mathematical foundations** of Voronoi diagrams including the formal definition and key properties

2. **Computational construction** using SciPy's implementation of Fortune's algorithm with $O(n \log n)$ complexity

3. **Visualization techniques** including:
   - Basic edge-based visualization
   - Colored finite regions with polygon clipping
   - Voronoi-Delaunay duality demonstration

4. **Geometric analysis** of cell properties (area, perimeter, edge count)

5. **Verification** of the fundamental nearest-neighbor property

The Voronoi diagram remains one of the most elegant and useful structures in computational geometry, with applications spanning computer graphics, spatial analysis, materials science, and beyond.